In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
spark

In [4]:
df = spark.read.csv('olist_classified_public_dataset.csv', inferSchema=True, header=True)

In [5]:
df.select('order_purchase_timestamp').show(1,truncate= False)

+--------------------------+
|order_purchase_timestamp  |
+--------------------------+
|2017-08-30 11:41:01.000000|
+--------------------------+
only showing top 1 row



In [62]:
import datetime
def timestr_datetime(timestampstr):
    return datetime.datetime.strptime(timestampstr, 'YYYY-MM-DD H24:mi:ss.ssss')

timestr_datetime('2017-08-30 11:41:01')

ValueError: time data '2017-08-30 11:41:01' does not match format 'YYYY-MM-DD H24:mi:ss.ssss'

In [55]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

udf_date_time = udf(timestr_datetime, TimestampType())

spark.udf.register('udf_dt', timestr_datetime)

<function __main__.timestr_datetime(timestampstr)>

In [23]:
def square_it(num):
    return num*num

square_it(9)

81

In [24]:
type(udf_date_time('2017-08-30 11:41:01.000000'))
udf_date_time(col('order_purchase_timestamp'))

Column<'udf_date_time(order_purchase_timestamp)'>

In [31]:
df1 = df.withColumn('ts', udf_date_time(col('order_purchase_timestamp')))

Traceback (most recent call last):
  File "F:\udemy_dgadraju_cca175\udemyCca175Env\lib\site-packages\pyspark\serializers.py", line 437, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "F:\udemy_dgadraju_cca175\udemyCca175Env\lib\site-packages\pyspark\cloudpickle\cloudpickle_fast.py", line 72, in dumps
    cp.dump(obj)
  File "F:\udemy_dgadraju_cca175\udemyCca175Env\lib\site-packages\pyspark\cloudpickle\cloudpickle_fast.py", line 540, in dump
    return Pickler.dump(self, obj)
TypeError: cannot pickle '_thread.RLock' object


PicklingError: Could not serialize object: TypeError: cannot pickle '_thread.RLock' object

In [32]:
df1.show()

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "F:\opt\spark-3.1.2-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\worker.py", line 604, in main
  File "F:\opt\spark-3.1.2-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\worker.py", line 596, in process
  File "F:\opt\spark-3.1.2-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\serializers.py", line 211, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "F:\opt\spark-3.1.2-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\serializers.py", line 132, in dump_stream
    for obj in iterator:
  File "F:\opt\spark-3.1.2-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\serializers.py", line 200, in _batched
    for item in iterator:
  File "F:\opt\spark-3.1.2-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\worker.py", line 450, in mapper
  File "F:\opt\spark-3.1.2-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\worker.py", line 450, in 

In [30]:
@udf(returnType=StringType())
def say_hello(name):
     return f"Hello {name}"
# This doesn't work anymore
#assert say_hello("Summer") == "Hello Summer"
df.withColumn("greetings", say_hello(col("order_status"))).show()

+---+---+------------+--------------------+-------------------+---------------+-----------------+------------------------+--------------------+-----------------------------+-----------------------------+--------------------+--------------+------------------------+---------------------+-------------------+--------------------------+------------------+------------+--------------------+----------------------+--------------------+-----------------------+---------------------+-------------+-----------------+------------+---------------------+----------------------+--------------------+-----------------+---------------+-------------------+--------------------+---------------+
|_c0| id|order_status|order_products_value|order_freight_value|order_items_qty|order_sellers_qty|order_purchase_timestamp|    order_aproved_at|order_estimated_delivery_date|order_delivered_customer_date|       customer_city|customer_state|customer_zip_code_prefix|product_category_name|product_name_lenght|product_descrip